In [1]:
import pandas as pd
import json
import os
import context_window_helper as cwh
import matplotlib.pyplot as plt


In [2]:
filepath = r"C:\Users\gluzk\source\repos\CS2_cheat_detection\Data_Conversion_Pipeline\test_data\0"

# Loading csv tick data as a pd.DataFrame
match_ticks = pd.read_parquet(path=filepath+".parquet")

# Loading json event data a list of tuples (str, pd.Dataframe)
def json_2_eventlist(filepath:str) -> list:   
    with open(filepath, "r") as f:
        json_data = json.load(f)

    data = []       

    for key, value in json_data.items():
        if isinstance(value, list):
            df = pd.DataFrame(value)
            data.append((key, df))

    return data

match_events = json_2_eventlist(filepath=filepath+".json")

print(match_events[47])

('cheaters',        name
0  Player_3
1  Player_4)


In [3]:
ticks_before_kill = 896
tick_after_kill = 128

context_window_vals = ["attacker_X", "attacker_Y", "attacker_Z", "attacker_vel_X", "attacker_vel_Y", "attacker_vel_Z", "attacker_pitch", "attacker_yaw", "attacker_pitch_delta", 
                       "attacker_yaw_delta", "attacker_pitch_head_delta", "attacker_yaw_head_delta", "attacker_flashed", "attacker_shot", "attacker_kill", "is_kill_through_smoke", 
                       "is_kill_wallbang", "attacker_midair", "attacker_weapon_knife", "attacker_weapon_auto_rifle", "attacker_weapon_semi_rifle", "attacker_weapon_pistol",
                       "attacker_weapon_grenade", "attacker_weapon_smg", "attacker_weapon_shotgun",
                       "victim_X", "victim_Y", "victim_Z", "victim_health", "victim_noise", "map_dust2", "map_mirage", "map_inferno", "map_train",
                       "map_nuke", "map_ancient", "map_vertigo", "map_anubis", "map_office", "map_overpass", "map_basalt", "map_edin", "map_italy", "map_thera", "map_mills"]

In [4]:
match_events

[('cs_round_final_beep',
       tick
  0    1441
  1    4904
  2   10784
  3   16702
  4   20264
  5   24997
  6   29665
  7   33505
  8   37399
  9   40430
  10  43919
  11  46860
  12  51273
  13  54060),
 ('player_footstep',
        tick user_name user_steamid
  0     1481  Player_6     Player_6
  1     1528  Player_7     Player_7
  2     1543  Player_6     Player_6
  3     1591  Player_7     Player_7
  4     1612  Player_6     Player_6
  ..     ...       ...          ...
  636  58132  Player_7     Player_7
  637  58724  Player_8     Player_8
  638  58780  Player_8     Player_8
  639  58842  Player_8     Player_8
  640  58901  Player_8     Player_8
  
  [641 rows x 3 columns]),
 ('player_connect_full',
      tick  user_name user_steamid
  0   9845  Player_10    Player_10
  1  37679   Player_6     Player_6),
 ('weapon_reload',
       tick  user_name user_steamid
  0    2086   Player_1     Player_1
  1    2163   Player_9     Player_9
  2    2757   Player_4     Player_4
  3    2898   P

In [5]:
# idx = match_ticks[(match_ticks["flash_duration"] > 3)&(match_ticks["name"] == "Player_3")].index
# vals = match_ticks.iloc[idx]["flash_duration"].tolist()
# print(len(vals))
# print(vals)

# plt.plot(match_ticks[match_ticks["name"] == "Player_3"]["flash_duration"])
# plt.show()
# match_ticks

# flash_duration
# flash_max_alpha

In [6]:
MDP = cwh.MatchDataProcessor(match_ticks)

In [ ]:
player = "Player_3" # For testug purposes, a cheater is specifically chosen
player_death_idx = -1


for idx, event in enumerate(match_events):
    if event[0] == "player_death":
        player_death_idx = idx
        break


player_deaths = MDP.get_player_kills(match_events, player, player_death_idx)
start_ticks, end_ticks = MDP.get_context_window_ticks(player_deaths, ticks_before_kill, tick_after_kill)

context_window = pd.DataFrame(columns=context_window_vals)

for i in range(len(start_ticks)):
    context_window["attacker_X"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "X")
    context_window["attacker_Y"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "Y")
    context_window["attacker_Z"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "Z")

    context_window["attacker_vel_X"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "velocity_X")
    context_window["attacker_vel_Y"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "velocity_Y")
    context_window["attacker_vel_Z"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "velocity_Z")

    context_window["attacker_pitch"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "pitch")
    context_window["attacker_yaw"] = MDP.get_tick_values(start_ticks[i], end_ticks[i], player, "yaw")
    context_window["attacker_pitch_delta"] = MDP.get_pitch_yaw_deltas("pitch", start_ticks[i], end_ticks[i], player)
    context_window["attacker_yaw_delta"] = MDP.get_pitch_yaw_deltas("yaw", start_ticks[i], end_ticks[i], player)

    context_window["attacker_pitch_head_delta"], context_window["attacker_yaw_head_delta"] = MDP.get_pitch_yaw_head_deltas(start_ticks[i], end_ticks[i], player, player_deaths.iloc[i]["user_name"])

    # TODO this flashed value should be verified ingame    
    context_window["attacker_flashed"] = MDP.get_is_player_flashed(start_ticks[i], end_ticks[i], player)

    break # Remove later, currently used for testing

context_window["attacker_flashed"]

0

In [8]:
# Testing to see if the delta to head values are correct. They seem to be, maybe should
# do a custom test scenario


# print(player_deaths.iloc[7]["weapon"])

# plt.plot(context_window["attacker_pitch_head_delta"])
# plt.axvline(x=896)
# plt.show()

# plt.plot(context_window["attacker_yaw_head_delta"])
# plt.axvline(x=896)
# plt.show()

# plt.plot(context_window["attacker_pitch"])
# plt.show()

# plt.plot(context_window["attacker_yaw"])
# plt.show()